#Considerando o ano de emissão como sendo 2019 e 2020, exiba as 3 siglas partidárias que mais gastaram cota parlamentar (valores líquidos) para trechos que contenham "POA" ou "FLN".

In [1]:
#importando sparksession
from pyspark.sql import SparkSession

#criando um objeto sparksession object e um appName 
spark = sparkSession=SparkSession.builder.appName("Pergunta5").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/01/18 23:35:09 WARN Utils: Your hostname, DESKTOP-6FAV8P8 resolves to a loopback address: 127.0.1.1; using 172.23.38.96 instead (on interface eth0)
24/01/18 23:35:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/18 23:35:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Fazendo a leitura do arquivo food_coded.csv
#df = spark.read.option("header", "true").csv("/Havan_Labs/Dados/cota-parlamentar.csv")

#Leitura de arquivo CSV
df= spark.read.format("csv").option("header", "true").load("/mnt/c/Havan_Labs/Dados/cota-parlamentar.csv")
df.show()

24/01/18 23:35:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+-------------------+------------+-----------+----------------+---------------------+------------+-------------+------+-----------------------+-------+------+----------+----------------+----------+---------+----+-----------------+--------------+--------------------+-------------------------+--------------------+---------+-------------+---------+------------+--------+----------+--------------+
|codlegislatura|         datemissao|idedocumento|idecadastro|indtipodocumento|nucarteiraparlamentar|nudeputadoid|nulegislatura|numano|numespecificacaosubcota|numlote|nummes|numparcela|numressarcimento|numsubcota|sgpartido|sguf|txnomeparlamentar|    txtcnpjcpf|        txtdescricao|txtdescricaoespecificacao|       txtfornecedor|txtnumero|txtpassageiro|txttrecho|vlrdocumento|vlrglosa|vlrliquido|vlrrestituicao|
+--------------+-------------------+------------+-----------+----------------+---------------------+------------+-------------+------+-----------------------+-------+------+-----

In [3]:
# utilizando funcoes do pyspark para filtro
from pyspark.sql.functions import *
# Filtrando os dados que nao sao nulos
resultdf_trechos = df.filter( (df.sgpartido.isNotNull()) & (df.vlrliquido.isNotNull() & ((df.numano == 2019) | (df.numano == 2020)) )).select('numano'
                                                        ,'sgpartido'
                                                        ,'txttrecho'
                                                        ,'vlrliquido')

# Adicionando uma coluna indicando se a palavra "TAXI" está presente nos campos específicos
resultdf_trechos_POA = resultdf_trechos.withColumn("contem_POA", (instr(upper(col("txttrecho")), "POA") > 0))

resultdf_trechos_POA_FLN = resultdf_trechos_POA.withColumn("contem_FLN", (instr(upper(col("txttrecho")), "FLN") > 0))

# Filtrando apenas os registros que contêm a palavra "TAXI"
resultdf_trechos_POA_FLN_filter = resultdf_trechos_POA_FLN.filter((col("contem_POA")) | (col("contem_FLN")))

# Exibindo o resultado
resultdf_trechos_POA_FLN_filter.show()


+------+---------+-----------+----------+----------+----------+
|numano|sgpartido|  txttrecho|vlrliquido|contem_POA|contem_FLN|
+------+---------+-----------+----------+----------+----------+
|  2020|       PT|    POA/BSB|    649.03|      true|     false|
|  2020|       PT|    POA/BSB|    485.03|      true|     false|
|  2020|       PT|    POA/BSB|    485.03|      true|     false|
|  2020|       PT|    BSB/POA|    647.57|      true|     false|
|  2020|       PT|    BSB/POA|    545.57|      true|     false|
|  2020|       PT|    BSB/POA|    745.57|      true|     false|
|  2020|      PDT|    FLN/POA|    263.03|      true|      true|
|  2020|      PDT|    POA/BSB|    514.03|      true|     false|
|  2020|      PDT|    POA/BSB|    355.03|      true|     false|
|  2020|      PDT|    POA/BSB|    355.03|      true|     false|
|  2020|      PDT|    BSB/POA|    600.57|      true|     false|
|  2020|      MDB|    POA/SDU|    981.03|      true|     false|
|  2020|       PT|    POA/BSB|   1112.03

In [4]:

resultdf_trechos_POA_FLN_filter_float = resultdf_trechos_POA_FLN_filter.withColumn('vlrliquido_float', col("vlrliquido").cast("float"))

In [5]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
resultdf_trechos_POA_FLN_filter_float.createOrReplaceTempView("tab_resultdf_trechos_POA_FLN_filter")

In [6]:

group_df = sparkSession.sql("select trechos.numano , trechos.sgpartido, sum(trechos.vlrliquido_float) soma_vlrliquido_float  \
                 from tab_resultdf_trechos_POA_FLN_filter trechos \
                 group by trechos.numano , trechos.sgpartido \
                 order by sum(trechos.vlrliquido_float) desc \
                 ")

In [7]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
group_df.createOrReplaceTempView("tab_resultdf_trechos_POA_FLN_filter_group")

In [9]:
df_final = sparkSession.sql("select * from tab_resultdf_trechos_POA_FLN_filter_group ").limit(3)

df_final.show()

+------+---------+---------------------+
|numano|sgpartido|soma_vlrliquido_float|
+------+---------+---------------------+
|  2019|       PT|    671757.1794242859|
|  2019|       PP|    653009.1891173124|
|  2019|      MDB|    620042.6010341644|
+------+---------+---------------------+

